In [164]:
%matplotlib

Using matplotlib backend: TkAgg


In [165]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from pysrc.simplertplot import queues
from hello.hello3 import HelloApp, TrueError
from time import time, sleep
import numpy as np
from matplotlib.ticker import NullFormatter, NullLocator, FormatStrFormatter
from collections import OrderedDict

In [166]:
#ip = '192.168.1.83'
ip = '71.189.82.196:85'
#ip = '71.189.82.196:83'

In [167]:
interval = 1000
minutes = 60
max_pts=minutes*60/(interval/1000)
max_pts=int(max_pts)
max_pts

3600

In [168]:
class MyPlotWidget(RTPlotWidget):
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self._special()

    def _update_data(self):
        self.line.set_data(self.x_data.get(), self.y_data.get())
        self.line2.set_data(self.x_data.get(), self.y_data2.get())
        self.line3.set_data(self.x_data.get(), self.y_data3.get())
        self.line4.set_data(self.x_data.get(), self.y_data4.get())
        self.subplot.relim()
        self.subplot.autoscale_view(True, True, True)
        self.ax2.relim()
        self.ax2.autoscale_view(True,True,True)
        lower, upper = self.subplot.get_ybound()
        self.subplot.set_ylim(lower-0.2, upper+0.2, True, None)
        lower, upper = self.ax2.get_ybound()
        self.ax2.set_ylim(lower-1, upper+1, True, None)
        self._update()
        
    def _special(self):
        self.ax2 = self.subplot.twinx()
        self.y_data2 = queues.RingBuffer(self.max_pts)
        self.line2, = self.ax2.plot(self.x_data.get(), self.y_data2.get(), 'g-')
        self.yaxis2 = self.ax2.yaxis
        self.yaxis2.set_major_formatter(FormatStrFormatter("%.2f"))
        
        self.y_data3 = queues.RingBuffer(self.max_pts)
        self.line3, = self.ax2.plot(self.x_data.get(), self.y_data3.get(), 'r-')
        
        self.y_data4 = queues.RingBuffer(self.max_pts)
        self.line4, = self.ax2.plot(self.x_data.get(), self.y_data4.get(), color='purple', ls="-")

    def extend_xy(self, x, y, y2, y3):
        self.x_data.extend(x)
        self.y_data.extend(y)
        self.y_data2.extend(y2)
        self.y_data3.xtend(y3)
        self._update_data()
        
    def legend(self, text1, text2, text3, text4, loc='upper right'):
        self.line.set_label(text1)
        self.line2.set_label(text2)
        self.line3.set_label(text3)
        self.line4.set_label(text4)

        h1, l1 = self.subplot.get_legend_handles_labels()
        h2, l2 = self.ax2.get_legend_handles_labels()
        h1.extend(h2)
        l1.extend(l2)
        self.subplot.legend(h1, l1, loc=loc)
        #self.subplot.legend(loc='upper right')
        #self.ax2.legend(loc='lower right')
        
    def reset(self):
        self.x_data.clear()
        self.y_data.clear()
        self.y_data2.clear()
        self.y_data3.clear()
        self.y_data4.clear()
        self._update_data()

    def add_xy(self, x, y, y2, y3, y4):
        self.x_data.put(x)
        self.y_data.put(y)
        self.y_data2.put(y2)
        self.y_data3.put(y3)
        self.y_data4.put(y4)
        self._update_data()

In [169]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [170]:
g_stop_tasks = False
after_ids = {}

def tk_task(tk, interval, f, aid=None):
    if aid is None:
        aid = id(f)
    def fn(*args, **kw):
        try:
            return f(*args, **kw)
        finally:
            if not g_stop_tasks:
                after_ids[aid] = tk.after(interval, fn)
    after_ids[aid] = tk.after(interval, fn)
    return aid

def task_cancel(aid):
    after_id = after_ids[aid]
    if after_id is not None:
        tk.after_cancel(after_id)

In [171]:
controller = 'do'

In [172]:
# import threading
# class Model():
#     def __init__(self, app, ctrl, interval=0.1):
#         self.mv = app.getMainValues()  # prepopulate
#         self.advv = app.getadvv()
#         self.time = get_time()
#         self.lock = threading.RLock()
#         self.running = threading.Event()
#         self.thread = None
#         self.app = app
#         self.interval = interval
#         self.ctrl = ctrl
#     def getvalues(self):
#         with self.lock:
#             pv = self.mv[self.ctrl]['pv']
#             op = self.advv['O2 Flow (%)']
#             op2 = self.advv["N2 Flow (%)"]
#             t = self.time
#         return t, pv, op, op2
#     def getMainValues(self):
#         return self.mv
#     def run_update(self):
#         self.running.set()
#         self.thread = threading.Thread(None, self._run_update, daemon=True)
#         self.thread.start()
#     def reconnect(self):
#         try:
#             self.app.reconnect()
#         except Exception:
#             pass
#     def _run_update(self):
#         while self.running.is_set():
#             self.time = get_time()
#             try:
#                 mv = self.app.getMainValues()
#             except Exception:
#                 self.reconnect()
#             else:
#                 with self.lock:
#                     self.mv = mv
#             try:
#                 adv = self.app.getadvv()
#             except Exception:
#                 self.reconnect()
#             else:
#                 with self.lock:
#                     self.advv = adv
                
#             sleep(self.interval)
#     def stop(self):
#         self.running.clear()
#         self.thread.join(timeout=10)
#         if self.thread.is_alive():
#             raise RuntimeError("Failed to stop thread")
#     def reset(self):
#         self.time = get_time()
#         self.stop()
#         self.run_update()

# h = HelloApp(ip)
# m = Model(h, controller, 0.5)
# m.run_update()
# yk = m.getvalues()[1]
# def filter_(pv):
#     global yk
#     yk += (pv - yk) / 4
#     return yk
    
# def update():
#     t, pv, op, op2 = m.getvalues()
#     #pv2 = filter_(pv)
#     p.add_xy(t, pv, op, op2)
#     update_labels(pv, op, op2)

In [173]:
import threading
class Model():
    def __init__(self, app, ctrl, interval=0.1):
        self.mv = app.getMainValues()  # prepopulate
        self.time = get_time()
        self.lock = threading.RLock()
        self.running = threading.Event()
        self.thread = None
        self.app = app
        self.interval = interval
        self.ctrl = ctrl
    def getvalues(self):
        with self.lock:
            pv = self.mv[self.ctrl]['pv']
            op = self.mv[self.ctrl]['outputUp']
            op2 = self.mv[self.ctrl]['outputDown']
            op3 = 100 - (self.mv['ph']['outputDown'] + op + op2)
            t = self.time
        return t, pv, op, op2, op3
    def getMainValues(self):
        return self.mv
    def run_update(self):
        self.running.set()
        self.thread = threading.Thread(None, self._run_update, daemon=True)
        self.thread.start()
    def _run_update(self):
        while self.running.is_set():
            try:
                mv = self.app.getMainValues()
            except Exception:
                self.app.reconnect()
            else:
                with self.lock:
                    self.mv = mv
                    self.time = get_time()
            sleep(self.interval)
    def stop(self):
        self.running.clear()
        self.thread.join(timeout=10)
        if self.thread.is_alive():
            raise RuntimeError("Failed to stop thread")
    def reset(self):
        self.time = get_time()
        self.stop()
        self.run_update()

h = HelloApp(ip, timeout=1, retry_count=1)
m = Model(h, controller, 0.5)
m.run_update()
yk = m.getvalues()[1]
def filter_(pv):
    global yk
    yk += (pv - yk) / 4
    return yk
    
def update():
    t, pv, *ops = m.getvalues()
    #pv2 = filter_(pv)
    p.add_xy(t, pv, *ops)
    update_labels(pv, *ops)

In [174]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [175]:
def conf2(ev):
    if ev.widget == root:
        other_height = sum(w.winfo_height() for w in (clbl, cbox, cout))
        p.tkcanvas.config(height=ev.height-other_height, width=ev.width-rbtn.winfo_width())

In [176]:
root = tk.Tk()
root.withdraw()

p = MyPlotWidget(root, max_pts=max_pts)
p.tkcanvas.config(height=400, width=600)
max_pts


3600

In [177]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
labels = OrderedDict()
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    labels[name] = l
    l.grid(row=row, column=2)
    
mk_label("PV", 0, 1)
mk_label("O2", 0, 2)
mk_label("N2", 0, 3)
mk_label("Air", 0, 4)
p.legend("PV", "O2", "N2", "Air")
        
def update_labels(*args):
    for a, l in zip(args, labels.values()):
        l.set_value(a)

    

In [178]:
p.grid(rowspan=20, column=0, row=0, columnspan=2)    
def reset():
    p.reset()
    global get_time
    get_time = timer().__next__
    m.reset()
rbtn = tk.Button(root, text="Reset", command=reset)
rbtn.grid(column=2, row=0, sticky="ew")
root.bind("<Configure>", conf2)
p.set_xaxis_text("Time (s)")
p.set_yaxis_text("%s PV" % controller.title())

clbl = tk.Label(root, text="Console:")
clbl.grid(row=21, column=0, sticky=tk.W)
cbox = tk.Entry(root)
cbox.grid(row=22, column=0, sticky=(tk.E, tk.W), columnspan=2)
cout = tk.Label(root)
cout.grid(row=23, column=0, sticky=(tk.E, tk.W), columnspan=2)

def exc_cmd(_):
    cmd = cbox.get()
    res = nores = object()
    err1 = None
    if cmd:
        try:
            try:
                res = eval(cmd, globals(), globals())
            except SyntaxError as e:
                err1 = e
            if err1:
                try:
                    exec(cmd, globals(), globals())
                except SyntaxError:
                    raise err1 from None
        except Exception:
            import traceback, io
            f = io.StringIO()
            traceback.print_exc(file=f)
            #print(f.getvalue())
            cout.config(text=f.getvalue())
        else:
            if res is not nores:
                msg = repr(cmd) + ": " + repr(res)
                print(msg)
                cout.config(text=msg)
            else:
                cout.config(text="")
        finally:
            cbox.delete(0, tk.END)
            
def exc_cmd2(_):
    cmd = cbox.get()
    exc_cmd_thread(cmd)
            
def exc_cmd_thread(cmd):
    res = None
    nores = object()
    def exc():
        nonlocal res
        err1 = None
        try:
            res = eval(cmd, globals(), globals())
        except SyntaxError as e:
            err1 = e
        except Exception:
            import traceback, io
            f = StringIO()
            traceback.print_exc(file=f)
            res = f.getvalue()
            return
        if err1:
            try:
                exec(cmd, globals(), globals())
            except SyntaxError:
                res = err1
    def done():
        if res is not nores:
            msg = repr(cmd) + ": " + repr(res)
            print(msg)
            cout.config(text=msg)
        else:
            cout.config(text="")
    def func():
        cbox.config(state='disabled')
        try:
            exc()
        finally:
            done()
        cbox.config(state='normal')
        cbox.delete(0, tk.END)
    threading.Thread(None, func, daemon=True).start()
        
            
        
cbox.bind("<Return>", exc_cmd2)

def on_closing():
    from tkinter import messagebox
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        global g_stop_tasks, g_after
        g_stop_tasks = True
        root.after_cancel(g_after)
        root.quit()
        root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.wm_deiconify()
tk_task(root, interval, update)
try:
    root.mainloop()
finally:
    m.stop()


bob
foo
"print('bob'); sleep(5); print('foo')": None


In [179]:
# import matplotlib.pyplot as plt
# import numpy as np
# np.random.seed(123456789)   # repeatable results
# f0 = 4
# t = np.arange(0,1.0,1.0/65536)
# mysignal = (np.mod(f0*t,1) < 0.5)*2.0-1
# mynoise = 1.0*np.random.randn(*mysignal.shape)
# plt.figure(figsize=(8,6))
# plt.plot(t, mysignal+mynoise, 'gray',
#          t, mysignal,         'black');
# delta_t = t[1]-t[0]

In [180]:
# mynoise2 = 0.1*np.random.randn(*mysignal.shape)
# alpha=delta_t/0.0005
# alpha=1/4
# def lpf(y, alpha):
#     y2 = np.zeros_like(y)
#     yk = y[0]
#     for i in range(len(y)):
#         yk += alpha * (y[i] - yk)
#         y2[i] = yk
#     return y2
# lpf_out2 = lpf(mysignal+mynoise2, alpha=alpha)
# fig=plt.figure(figsize=(8,10))
# ax=fig.add_subplot(3,1,1)
# ax.plot(t,mysignal+mynoise2,t,mysignal)
# ax.set_ylabel('original signal')
# ax2=fig.add_subplot(3,1,2)
# ax2.plot(t,lpf_out2)
# ax2.set_ylabel('lpf output')
# #ax3=fig.add_subplot(3,1,3)
# #ax3.plot(t,lpf_out2)
# #ax3.set_ylim(-1.05,-0.95)
# #ax3.set_ylabel('lpf output, zoom')